In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ROOT

Welcome to JupyROOT 6.22/02


In [2]:
root_file = ROOT.TFile("../../ML-Data/LAOD/True_Merged_Pi0.root", "OPEN")
tree_pi0 = root_file.Get("True_Merged_Calo_Cluster_Pi0")

In [3]:
nentries = tree_pi0.GetEntries()
cluster = np.zeros((50,50), dtype=np.float32)
tree_pi0.SetBranchAddress("Cluster", cluster)
data, col = tree_pi0.AsMatrix(columns=['ClusterType','E', 'Pt', 'ModuleNumber', 'X', 'Y'], return_labels=True)
pi0_df = pd.DataFrame(data=data, columns=col)
print(f"Entries: {nentries}")
print(f"Shape of cluster: {cluster.shape}")
pi0_df.head()

Entries: 32
Shape of cluster: (50, 50)


,ClusterType,E,Pt,ModuleNumber,X,Y
0,1.0,12.739141,12.245251,9.0,1.0,7.0
1,2.0,5.609921,5.194993,9.0,18.0,21.0
2,2.0,5.168575,4.748755,9.0,21.0,14.0
3,2.0,5.752418,5.277642,9.0,22.0,21.0
4,2.0,6.982239,6.694414,8.0,44.0,10.0


In [27]:
for i in range(nentries):
    tree_pi0.GetEntry(i)

Error R__unzip_header: error in header.  Values: 00
Error in <TBasket::ReadBasketBuffers>: Inconsistency found in header (nin=0, nbuf=0)
Error in <TBasket::ReadBasketBuffers>: fNbytes = 35641, fKeylen = 0, fObjlen = 41119, noutot = 0, nout=0, nin=0, nbuf=0
Error R__unzip_header: error in header.  Values: 00
Error in <TBasket::ReadBasketBuffers>: Inconsistency found in header (nin=0, nbuf=0)
Error in <TBasket::ReadBasketBuffers>: fNbytes = 35641, fKeylen = 0, fObjlen = 41119, noutot = 0, nout=0, nin=0, nbuf=0
Error R__unzip_header: error in header.  Values: 00
Error in <TBasket::ReadBasketBuffers>: Inconsistency found in header (nin=0, nbuf=0)
Error in <TBasket::ReadBasketBuffers>: fNbytes = 35641, fKeylen = 0, fObjlen = 41119, noutot = 0, nout=0, nin=0, nbuf=0
Error R__unzip_header: error in header.  Values: 00
Error in <TBasket::ReadBasketBuffers>: Inconsistency found in header (nin=0, nbuf=0)
Error in <TBasket::ReadBasketBuffers>: fNbytes = 35641, fKeylen = 0, fObjlen = 41119, noutot

In [17]:
croot_file = ROOT.TFile("../../ML-Data/LAOD/GammaCaloMergedTest_119.root", "OPEN")

In [22]:
tree = croot_file.Get("True_Merged_Calo_Cluster_Pi0")